In [14]:
from dotenv import dotenv_values
config = dotenv_values(".env")

In [15]:
import pandas as pd
import psycopg2
conn = psycopg2.connect(dbname=config['DBNAME'], user=config['USER'], password=config['PASSWORD'])

In [16]:
df_morticd = pd.read_sql('SELECT * from formatted.morticd_7_22b4_1642928616;', conn)
df_countries = pd.read_sql('SELECT * from formatted.countrycodes_0_8c41_1642928619;', conn)

In [17]:
# Check if morticd countries are in countrycode country
df_morticd['CountryWrongCode'] = df_morticd['Country'].apply(lambda x: x in df_countries.Country)

In [18]:
# Check Size
df_morticd['CountryWrongSize'] = df_morticd['Country'].apply(lambda x: x != None and len(str(x))  != 4)
df_morticd['Admin1WrongSize'] = df_morticd['Admin1'].apply(lambda x: x != None and len(str(x)) != 3)
df_morticd['SubDivWrongSize'] = df_morticd['SubDiv'].apply(lambda x: x != None and len(str(x)) != 3)
df_morticd['YearWrongSize'] = df_morticd['Year'].apply(lambda x: x != None and len(str(x)) != 4)
df_morticd['ListWrongSize'] = df_morticd['List'].apply(lambda x: x != None and len(str(x)) != 3)
df_morticd['CauseWrongSize'] = df_morticd['Cause'].apply(lambda x: x != None and len(str(x)) > 4)
df_morticd['SexWrongSize'] = df_morticd['Sex'].apply(lambda x: x != None and len(str(x)) != 1)
df_morticd['FrmatWrongSize'] = df_morticd['Frmat'].apply(lambda x: x != None and len(str(x)) != 2)
df_morticd['IM_FrmatWrongSize'] = df_morticd['IM_Frmat'].apply(lambda x: x != None and len(str(x)) != 2)

In [19]:
names = df_morticd.filter(regex=("Deaths*")).columns
for death in names:    
    df_morticd[death + 'WrongSize'] = df_morticd[death].apply(lambda x: len(str(x)) > 8)
   

In [20]:
df_morticd

,Country,Admin1,SubDiv,Year,List,Cause,Sex,Frmat,IM_Frmat,Deaths1,...,Deaths21WrongSize,Deaths22WrongSize,Deaths23WrongSize,Deaths24WrongSize,Deaths25WrongSize,Deaths26WrongSize,IM_Deaths1WrongSize,IM_Deaths2WrongSize,IM_Deaths3WrongSize,IM_Deaths4WrongSize
0,1125,None,None,1955,07A,A000,1,01,08,107822,...,False,False,False,False,False,False,False,False,False,False
1,1125,None,None,1955,07A,A000,2,01,08,105750,...,False,False,False,False,False,False,False,False,False,False
2,1125,None,None,1955,07A,A001,1,01,08,1302,...,False,False,False,False,False,False,False,False,False,False
3,1125,None,None,1955,07A,A001,2,01,08,739,...,False,False,False,False,False,False,False,False,False,False
4,1125,None,None,1955,07A,A002,1,01,08,69,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281744,4340,None,None,1960,07B,202,2,02,08,4,...,False,False,False,False,False,False,False,False,False,False
281745,4340,None,None,1960,07B,203,1,02,08,13,...,False,False,False,False,False,False,False,False,False,False
281746,4340,None,None,1960,07B,203,2,02,08,13,...,False,False,False,False,False,False,False,False,False,False
281747,4340,None,None,1960,07B,205,1,02,08,0,...,False,False,False,False,False,False,False,False,False,False


In [21]:
wrongs = df_morticd.filter(regex=(".Wrong*")).columns
wrongs

Index(['CountryWrongCode', 'CountryWrongSize', 'Admin1WrongSize',
       'SubDivWrongSize', 'YearWrongSize', 'ListWrongSize', 'CauseWrongSize',
       'SexWrongSize', 'FrmatWrongSize', 'IM_FrmatWrongSize',
       'Deaths1WrongSize', 'Deaths2WrongSize', 'Deaths3WrongSize',
       'Deaths4WrongSize', 'Deaths5WrongSize', 'Deaths6WrongSize',
       'Deaths7WrongSize', 'Deaths8WrongSize', 'Deaths9WrongSize',
       'Deaths10WrongSize', 'Deaths11WrongSize', 'Deaths12WrongSize',
       'Deaths13WrongSize', 'Deaths14WrongSize', 'Deaths15WrongSize',
       'Deaths16WrongSize', 'Deaths17WrongSize', 'Deaths18WrongSize',
       'Deaths19WrongSize', 'Deaths20WrongSize', 'Deaths21WrongSize',
       'Deaths22WrongSize', 'Deaths23WrongSize', 'Deaths24WrongSize',
       'Deaths25WrongSize', 'Deaths26WrongSize', 'IM_Deaths1WrongSize',
       'IM_Deaths2WrongSize', 'IM_Deaths3WrongSize', 'IM_Deaths4WrongSize'],
      dtype='object')

In [22]:
for wrong_col in wrongs:
   if len(df_morticd[df_morticd[wrong_col] == True]) > 0:
      print(wrong_col)

In [ ]:
df_demo_country = pd.read_sql('SELECT * from formatted.DemographicCountry;', conn)
df_demo_labels = pd.read_sql('SELECT * from formatted.DemographicLabels;', conn)
df_demo_indic = pd.read_sql('SELECT * from formatted.Demographic;', conn)

In [ ]:
# Check if demographic country codes are in demographic country table
df_demo_indic['CountryId'] = df_demo_indic['CountryId'].apply(lambda x: x in df_demo_country.Code)

In [ ]:
# Check if demographic Indicator ID are in demographic label table
df_demo_indic['IndicatorId'] = df_demo_indic['Indicator'].apply(lambda x: x in df_demo_labels.Indicator)

In [ ]:
# Check size Demographic label columns
df_demo_labels['IndicatorWrongSize'] = df_demo_labels['Indicator'].apply(lambda x: x != None and len(str(x))  != 50)
df_demo_labels['DescriptionWrongSize'] = df_demo_labels['Description'].apply(lambda x: x != None and len(str(x))  != 100)

In [ ]:
# Check size Demographic country columns
df_demo_country['Code'] = df_demo_country['Code'].apply(lambda x: x != None and len(str(x))  != 3)
df_demo_country['Name'] = df_demo_country['Name'].apply(lambda x: x != None and len(str(x))  != 100)

In [ ]:
# Check size Demographic country columns
df_demo_indic['IndicatorId'] = df_demo_indic['IndicatorId'].apply(lambda x: x != None and len(str(x))  != 50)
df_demo_indic['CountryId'] = df_demo_indic['CountryId'].apply(lambda x: x != None and len(str(x))  != 3)
df_demo_indic['Year'] = df_demo_indic['Year'].apply(lambda x: x != None and len(str(x))  != 4)
df_demo_indic['Magnitude'] = df_demo_indic['Magnitude'].apply(lambda x: x != None and len(str(x))  != 50)
df_demo_indic['Qualifier'] = df_demo_indic['Qualifier'].apply(lambda x: x != None and len(str(x))  != 50)

In [ ]:
# Check Uniques Demographic Country
df_demo_country_unique = pd.DataFrame(df_demo_country['Code'].value_counts())
df_demo_country['CodeWrongUnique'] = df_demo_country_unique['Code'] > 2

In [ ]:
# Check Uniques Demographic Labels
df_demo_labels_unique = pd.DataFrame(df_demo_labels['Indicator'].value_counts())
df_demo_labels['IndicatorWrongUnique'] = df_demo_labels_unique['Indicator'] > 2

In [ ]:
# Check Uniques Country codes
df_country_code_unique = pd.DataFrame(df_countries['Country'].value_counts())
df_countries['CountryWrongUnique'] = df_country_code_unique['Indicator'] > 2

In [ ]:
# Check possitive values